In [1]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [3]:
auth.authenticate_user()

In [4]:
project_id='nus-datathon-2018-team-01'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [5]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

Sweet! We are interested in Sp02 (row 1) and 02 saturation pulseoxymetry (row 3). 
They both refer to the same concept, but they have different names because they come from different sources. 

## COHORT SELECTION

We want patients who  are:

*   16 or over at the start of the stay.
*   Min 72 hours in ICU
*   Only first admission
*   Recieved supplemental oxygen TODO

### run the code below in BigQuery and save it as a view 

patient_cohort:





In [0]:
run_query('''
WITH ps AS (
SELECT
  icu.subject_id,
  icu.hadm_id,
  icu.icustay_id,
  pat.dob,
  DATE(icu.intime) AS icu_date,
  DATETIME_DIFF(icu.outtime, icu.intime, HOUR) / 24 AS icu_length_of_stay,
  DATE_DIFF(DATE(icu.intime), DATE(pat.dob), YEAR) AS age
FROM `physionet-data.mimiciii_clinical.icustays` AS icu
INNER JOIN `physionet-data.mimiciii_clinical.patients` AS pat
  ON icu.subject_id = pat.subject_id),
first_stay AS (
SELECT
MIN(DATE(icu.intime)) AS first_icu_date,
icu.subject_id AS subject_id
FROM `physionet-data.mimiciii_clinical.icustays` AS icu
GROUP BY subject_id),
ventilation AS (
SELECT
  MAX(chart.valuenum) AS max_fiO2,
  chart.subject_id,
  chart.icustay_id
FROM `physionet-data.mimiciii_clinical.chartevents` AS chart
WHERE chart.itemid in (3420, 190, 223835, 3422)
GROUP BY chart.subject_id, chart.icustay_id)
SELECT
  ps.subject_id AS subject_id,
  MAX(ps.icustay_id) AS icustay_id,
  MAX(ps.age) as age,
  MAX(ps.icu_length_of_stay) as icu_length_of_stay,
  MAX(ventilation.max_fiO2) as max_fiO2
FROM ps
INNER JOIN first_stay
  ON ps.subject_id = first_stay.subject_id
INNER JOIN ventilation
  ON ps.subject_id = ventilation.subject_id
WHERE age >= 16
AND icu_length_of_stay >= 3
AND first_stay.first_icu_date = icu_date
AND ventilation.max_fiO2 > 24
GROUP BY ps.subject_id
''')

In [14]:
run_query('''
SELECT * FROM oxygenation.patient_cohort
''')

df.head()

    subject_id  icustay_id  age  icu_length_of_stay  max_fiO2
0        44228      217992   58                   4      40.0
1        42281      256746   68                   4     100.0
2        42066      244243   51                   5      50.0
3        42199      274509   73                   5     100.0
4        43798      243229   62                   7     100.0
5        40595      276601   76                   9     100.0
6        41795      216185   49                  10     100.0
7        41795      216185   49                  10      45.0
8        41914      256338   55                  13      70.0
9        42075      298685   80                  14     100.0
10       40310      210989   41                  19     100.0
11       40310      210989   41                  19     100.0
12       42367      250305   87                  29     100.0
13       44212      239396   45                  31     100.0


## ICD CODES!





In [0]:
run_query('''
WITH icd_presence AS (
SELECT
icd.subject_id,
pat.icustay_id,
SAFE_CAST(SUBSTR(icd.icd9_code, 0, 3) as numeric) AS icd_num
FROM `physionet-data.mimiciii_clinical.diagnoses_icd` AS icd
INNER JOIN `nus-datathon-2018-team-01.oxygenation.patient_cohort` AS pat
  ON pat.subject_id = icd.subject_id)
SELECT
icd_presence.subject_id,
icd_presence.icustay_id,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 001 AND 139 THEN 1 END) > 0 AS has_infectous_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 140 AND 239 THEN 1 END) > 0 AS has_neoplasm_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 240 AND 279 THEN 1 END) > 0 AS has_endocrine_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 280 AND 289 THEN 1 END) > 0 AS has_blood_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 290 AND 319 THEN 1 END) > 0 AS has_mental_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 320 AND 389 THEN 1 END) > 0 AS has_nervous_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 390 AND 459 THEN 1 END) > 0 AS has_circulatory_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 460 AND 519 THEN 1 END) > 0 AS has_respiratory_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 520 AND 579 THEN 1 END) > 0 AS has_digestive_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 580 AND 629 THEN 1 END) > 0 AS has_urinary_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 630 AND 679 THEN 1 END) > 0 AS has_pregnancy_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 680 AND 709 THEN 1 END) > 0 AS has_skin_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 710 AND 739 THEN 1 END) > 0 AS has_muscle_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 740 AND 759 THEN 1 END) > 0 AS has_congenital_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 760 AND 779 THEN 1 END) > 0 AS has_perinatal_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 780 AND 799 THEN 1 END) > 0 AS has_other_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 800 AND 999 THEN 1 END) > 0 AS has_injury_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 410 AND 414 THEN 1 END) > 0 AS has_isachaemic_heart_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 427 AND 427 THEN 1 END) > 0 AS has_atrial_fibrillation_disease,
COUNT(CASE WHEN icd_presence.icd_num BETWEEN 434 AND 434 THEN 1 END) > 0 AS has_stroke_disease
FROM icd_presence
GROUP BY icd_presence.subject_id, icd_presence.icustay_id
```)


## Fluid Balance


In [0]:
run_query('''-- fluid intake in table ‘inputevents_mv’

WITH intake_im AS (

  SELECT afc.*

         , im.starttime

         , im.endtime

        -- convert the unit to ml

         , CASE WHEN im.totalamountuom like 'uL' THEN im.totalamount/1000

                WHEN im.totalamountuom like 'ounces' THEN im.totalamount*29.27

                ELSE im.totalamount END AS amount

    FROM `physionet-data.mimiciii_clinical.icustay_detail` AS afc

         LEFT JOIN `physionet-data.mimiciii_clinical.inputevents_mv` AS im

          ON im.icustay_id = afc.icustay_id

          WHERE (im.totalamountuom like 'ml' or im.totalamountuom like 'uL' or im.totalamountuom like 'ounces')

   AND im.starttime BETWEEN afc.intime AND DATETIME_ADD(afc.intime, INTERVAL 72 HOUR)

     AND amount IS NOT NULL

)





--fluid intake in table ‘inputevents_cv’

-- inputevents_cv only has charttime

, intake_ic AS (

  SELECT afc.*

         , ic.charttime

         -- convert the unit to ml

         , CASE WHEN ic.amountuom like 'tsp' THEN amount/5

                              ELSE ic.amount END AS amount

    FROM `physionet-data.mimiciii_clinical.icustay_detail` afc

         LEFT JOIN `physionet-data.mimiciii_clinical.inputevents_cv` ic

           ON ic.icustay_id = afc.icustay_id

          WHERE (ic.amountuom like 'cc' or ic.amountuom like 'ml' or ic.amountuom like 'tsp')

   AND ic.charttime BETWEEN afc.intime AND DATETIME_ADD(afc.intime, INTERVAL 72 HOUR)

)



--fluid output in table ‘outputevents’

  , output AS (

  SELECT afc.icustay_id

       --, value

       --, oe.itemid

       --, di.label

       , sum(value) AS output_fluid

  FROM `physionet-data.mimiciii_clinical.icustay_detail` afc

       LEFT JOIN `physionet-data.mimiciii_clinical.outputevents` oe

       ON afc.icustay_id = oe.icustay_id

       LEFT JOIN `physionet-data.mimiciii_clinical.d_items` di

       ON di.itemid = oe.itemid

 WHERE oe.charttime BETWEEN afc.intime AND DATETIME_ADD(afc.intime, INTERVAL 72 HOUR)

 GROUP BY afc.icustay_id

)



-- sum of fluid input in 72 hours

  , input AS (

  SELECT intake.icustay_id

       , sum(intake.amount) AS intake_fluid

  FROM (SELECT icustay_id

               , amount

          from intake_im

        UNION DISTINCT

        SELECT icustay_id

               , amount

          from intake_ic) intake

 GROUP BY intake.icustay_id

)



SELECT DISTINCT input.icustay_id

       --, output.output_fluid

       , input.intake_fluid - output.output_fluid AS fluid_balance

  FROM input

       LEFT JOIN output

       ON output.icustay_id = input.icustay_id

--Delete one wrong record

 WHERE output_fluid <> 150400'''

## Tidal High Count (Invasive Inventilation)

In [0]:
run_query('''set SEARCH_PATH TO mimiciii;

-- choose ventilation records recorded in first 3 days
-- and if the endtime exceeds ICU intime + 24 hours, change the endtime to 'intime + 24 hours'
WITH vent_cohort AS (
  SELECT vd.icustay_id
        , icud.intime
        , vd.starttime
        , CASE WHEN vd.endtime > icud.intime + INTERVAL '24' HOUR
                 THEN (icud.intime + INTERVAL '24' HOUR)
                 ELSE vd.endtime
          END
  FROM icustay_detail icud
       LEFT JOIN ventdurations vd
       ON vd.icustay_id = icud.icustay_id
       AND vd.starttime BETWEEN icud.intime AND icud.intime + INTERVAL '1' day
)

  , tidal AS (
SELECT ce.icustay_id
       , (ce.value::NUMERIC)/(echo.weight::NUMERIC) AS tidal_volume_per_weight
       --, ce.valueuom
       --, echo.weight
  FROM chartevents ce
  LEFT JOIN echo_categorized echo
       ON echo.hadm_id::text = ce.hadm_id::text
  LEFT JOIN vent_cohort vc
       ON ce.icustay_id = vc.icustay_id
 WHERE ce.itemid = ANY (ARRAY[681, 682, 224685]) -- these itemid are for tidal volume
       AND echo.weight IS NOT NULL -- some guys didn't have these records, just remove them
       AND echo.weight !~~ 'None' -- text in 'weight'
       AND ce.charttime BETWEEN vc.starttime AND vc.endtime  -- only select tidal volume recorded during invasive ventilation
)
, tidal_total AS (
  SELECT tidal.*
         , CASE WHEN tidal.tidal_volume_per_weight >= 6.5 THEN 1 ELSE 0 END AS tidal_label
         , tidal_total.total_count
        FROM tidal
             LEFT JOIN (
                SELECT icustay_id
                       , count(icustay_id) AS total_count
                  FROM tidal
                 GROUP BY icustay_id
            ) tidal_total
             ON tidal_total.icustay_id = tidal.icustay_id
)

  , tidal_risk AS (
  SELECT tt.icustay_id
       , tt.total_count
       , sum(tt.tidal_label) AS tidal_high_count
    FROM tidal_total tt
   GROUP BY tt.icustay_id, tt.total_count
)

SELECT icustay_id
      , tidal_high_count
      , total_count
       , tidal_high_count::NUMERIC /total_count AS tidal_high_count2
  FROM tidal_risk
  ORDER BY tidal_high_count DESC'''

### Mechanical Ventilative Volume


In [0]:
run_query('''-- choose ventilation records recorded in first 3 days
-- and if the endtime exceeds ICU intime + 24 hours, change the endtime to 'intime + 24 hours'
WITH vent_cohort AS (
 SELECT vd.icustay_id
    , icud.intime
    , vd.starttime
    , CASE WHEN vd.endtime > DATETIME_ADD(icud.intime, INTERVAL 24 HOUR)
         THEN DATETIME_ADD(icud.intime, INTERVAL 24 HOUR)
         ELSE vd.endtime
     END AS endtime
 FROM `physionet-data.mimiciii_clinical.icustay_detail` icud
    LEFT JOIN `physionet-data.mimiciii_clinical.ventdurations` vd
    ON vd.icustay_id = icud.icustay_id
 WHERE vd.starttime BETWEEN icud.intime AND DATETIME_ADD(icud.intime, INTERVAL 1 day)
)
, weight AS (
SELECT icustay_id
    , AVG(weight) AS weight
 FROM `physionet-data.mimiciii_clinical.weightdurations`
 GROUP BY icustay_id
)
 , tidal AS (
SELECT ce.icustay_id
    , safe_CAST(ce.value as FLOAT64) /wt.weight AS tidal_volume_per_weight
    --, ce.valueuom
    --, echo.weight
 FROM `physionet-data.mimiciii_clinical.chartevents` ce
 LEFT JOIN weight wt
    ON wt.icustay_id = ce.icustay_id
 LEFT JOIN vent_cohort vc
    ON ce.icustay_id = vc.icustay_id
 WHERE ce.itemid IN (681, 682, 224685) -- these itemid are for tidal volume
    AND wt.weight IS NOT NULL -- some guys didn't have these records, just remove them
    AND ce.charttime BETWEEN vc.starttime AND vc.endtime -- only select tidal volume recorded during invasive ventilation
)
, tidal_total AS (
 SELECT tidal.*
     , CASE WHEN tidal.tidal_volume_per_weight >= 6.5 THEN 1 ELSE 0 END AS tidal_label
     , tidal_total.total_count
    FROM tidal
       LEFT JOIN (
        SELECT icustay_id
            , count(icustay_id) AS total_count
         FROM tidal
         GROUP BY icustay_id
      ) tidal_total
       ON tidal_total.icustay_id = tidal.icustay_id
)
 , tidal_risk AS (
 SELECT tt.icustay_id
    , tt.total_count
    , sum(tt.tidal_label) AS tidal_high_count
  FROM tidal_total tt
  GROUP BY tt.icustay_id, tt.total_count
)
SELECT icustay_id
   , tidal_high_count
   , total_count
    , tidal_high_count/total_count AS tidal_high_count2
 FROM tidal_risk
 ORDER BY tidal_high_count DESC'''

# Final Queries

## patients_Resutls
We will run a few codes over a couple of steps for computational ease 

### Step 1 


In [16]:
run_query(''' 
SELECT DISTINCT 

                HADM.HADM_id,                 
                PC.subject_id as patient_ID,
                P.gender as gender,
                PC.age as age,  
              PC.icu_length_of_stay,
  CASE 
  WHEN HADM.deathtime BETWEEN HADM.admittime and HADM.dischtime
            THEN 1
  ELSE 0
  END AS mortality_in_Hospt, 
    CASE 
  WHEN HADM.deathtime BETWEEN icu.intime and icu.outtime
            THEN 1
  ELSE 0
  END AS mortality_in_ICU,
  HADM.deathtime as deathtime, 
  ICU.intime as ICU_intime, 
                ICD.*
  
FROM `nus-datathon-2018-team-01.oxygenation.patient_cohort` PC
INNER JOIN `physionet-data.mimiciii_clinical.patients` P 
  ON P.subject_id = PC.subject_id
INNER JOIN `physionet-data.mimiciii_clinical.admissions` HADM
  ON PC.subject_id = HADM.subject_id
INNER JOIN `physionet-data.mimiciii_clinical.icustays` ICU
  ON PC.subject_id = ICU.subject_id
  AND HADM.HADM_ID = ICU.HADM_ID
INNER JOIN `physionet-data.mimiciii_clinical.chartevents` C
  ON C.subject_id = PC.subject_id 
    AND C.HADM_id = HADM.HADM_id 
    AND C.icustay_id = ICU.icustay_id 
    AND PC.icustay_id = ICU.icustay_id
INNER JOIN `nus-datathon-2018-team-01.oxygenation.icd_codes` ICD 
  ON ICD.subject_id = PC.subject_id 
  AND ICU.icustay_id = ICD.icustay_id
WHERE C.ITEMID in (220277, 646) 
AND C.valuenum IS NOT NULL; 

''')



### step 2 

In [0]:
 run_query('''SELECT DISTINCT PRI1.*, 
                  apsiii.apsiii,
                  elix.congestive_heart_failure, 
                  elix.hypertension, 
                  elix.chronic_pulmonary, 
                  elix.diabetes_uncomplicated, 
                  elix.diabetes_complicated, 
                  elix.renal_failure, 
                  elix.liver_disease, 
                  elix.lymphoma, 
                  elix.solid_tumor, 
                  elix.metastatic_cancer,
                  angus.angus
FROM `nus-datathon-2018-team-01.oxygenation1.patients_Results_Intermediate_1` PRI1
INNER JOIN `physionet-data.mimiciii_clinical.elixhauser_quan` AS elix
  ON elix.HADM_ID = PRI1.HADM_ID
INNER JOIN `physionet-data.mimiciii_clinical.angus_sepsis` AS angus
  ON angus.hadm_id = PRI1.hadm_id
INNER JOIN `physionet-data.mimiciii_clinical.apsiii` AS apsiii
  ON apsiii.hadm_id = PRI1.hadm_id
  AND apsiii.subject_id = PRI1.patient_id
  AND apsiii.icustay_id = PRI1.icustay_id''')

### step 3

In [0]:
run_query('''SELECT
      PIR2.*,  
      CASE 
      (SELECT count(mechvent)
       FROM `physionet-data.mimiciii_clinical.ventsettings` VS
       WHERE mechvent = 1 
       AND VS.icustay_id = PIR2.icustay_id)  
       WHEN 0 THEN 0 
       ELSE 1 
       END AS invasive
FROM `nus-datathon-2018-team-01.oxygenation1.patients_Results_Intermediate_2` PIR2 ''')

### stetp 4 

In [1]:
run_query('''SELECT PIR3.*, 
       FB.fluid_balance 
FROM `nus-datathon-2018-team-01.oxygenation1.patients_Results_Intermediate_3` PIR3 
LEFT OUTER JOIN `nus-datathon-2018-team-01.oxygenation1.fluid_balance` FB 
  ON PIR3.icustay_id = FB.icustay_id;)

SyntaxError: ignored

###step 4 

In [0]:
run_query('''SELECT PIR4.*, 
       SOFA.sofa
FROM `nus-datathon-2018-team-01.oxygenation1.patients_Results_Intermediate_4` PIR4
LEFT OUTER JOIN `physionet-data.mimiciii_clinical.sofa` SOFA 
ON PIR4.patient_id = SOFA.subject_id 
AND PIR4.icustay_id = SOFA.icustay_id) ; 

### step 5

In [0]:
run_query('''SELECT PIR5.*, 
       MVV.tidal_high_count2 as tidal_count_percentage
FROM `nus-datathon-2018-team-01.oxygenation1.patients_Results_Intermediate_5` PIR5
LEFT OUTER JOIN `nus-datathon-2018-team-01.oxygenation1.mechanical_ventilative_volume` MVV 
ON PIR5.icustay_id = MVV.icustay_id);

### step 6

In [0]:
run_query('''SELECT DISTINCT OR6.patient_id
      , max(icus.first_careunit) as first_care_unit
 from `nus-datathon-2018-team-01.oxygenation1.patients_Results_Intermediate_6` as or6
      left outer join `physionet-data.mimiciii_clinical.icustays`  as icus
      on icus.icustay_id = or6.icustay_id
where icus.first_careunit is not null
group by OR6.patient_id)

## measurements_Results

In [0]:
run_query(''' 
SELECT DISTINCT PC.subject_id as patient_ID,
              C.valuenum as spO2_Value, 
              C.valueuom as sp02_Unit,
              C.charttime as measurement_time
              
FROM `nus-datathon-2018-team-01.oxygenation.patient_cohort` PC
INNER JOIN `physionet-data.mimiciii_clinical.patients` P 
  ON P.subject_id = PC.subject_id
INNER JOIN `physionet-data.mimiciii_clinical.admissions` HADM
  ON PC.subject_id = HADM.subject_id
INNER JOIN `physionet-data.mimiciii_clinical.icustays` ICU
  ON PC.subject_id = ICU.subject_id
  AND HADM.HADM_ID = ICU.HADM_ID
INNER JOIN `physionet-data.mimiciii_clinical.chartevents` C
  ON C.subject_id = PC.subject_id 
    AND C.HADM_id = HADM.HADM_id 
    AND C.icustay_id = ICU.icustay_id 
    AND PC.icustay_id = ICU.icustay_id
INNER JOIN `nus-datathon-2018-team-01.oxygenation.icd_codes` ICD 
  ON ICD.subject_id = PC.subject_id 
  AND ICU.icustay_id = ICD.icustay_id
WHERE C.ITEMID in (220277, 646) 
AND C.valuenum IS NOT NULL; 
''')
